---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [2]:
def data_etl():
    train = pd.read_csv('readonly/train.csv', encoding='latin1', parse_dates = ['ticket_issued_date', 'hearing_date'], low_memory=False)
    test = pd.read_csv('readonly/test.csv', parse_dates = ['ticket_issued_date', 'hearing_date'])
    latlong = pd.read_csv('readonly/latlons.csv')
    addresses = pd.read_csv('readonly/addresses.csv')

    # Corregimos las latitudes y longitudes faltantes
    # %pip install geopy
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="data_course")
    df_null = latlong[(latlong.lat.isnull()) & (latlong.lon.isnull())]
    addresses_null = df_null.address.values
    index_null = df_null.index.values
    adresses_correct = [adress.split('MI')[0] for adress in addresses_null]

    for i in range(0,len(adresses_correct)):
        location = geolocator.geocode(adresses_correct[i])
        latlong.loc[latlong.index==index_null[i], 'lat'] = location.latitude
        latlong.loc[latlong.index==index_null[i], 'lon'] = location.longitude
    
    # selecciono las columnas que no estan en test y las elimino de train manteniendo 'compliance
    test_columns = test.columns.values
    train_columns = train.columns.values

    list_columns = []
    for name in train_columns:
        if name not in test_columns:
            list_columns.append(name)
            
    list_columns.remove('compliance')
    train = train.drop(list_columns, axis=1)

    # Eliminamos los nulos de 'compliance'
    train.dropna(subset=['compliance'], inplace=True)

    # Hay filas en train que no pertenecen a USA.. hay que eliminarlos
    train = train[train.country=='USA']

    # Merge de addresses con las coordenadas para unificar y luego hacer merge con los df test y train
    adress_complete = pd.merge(addresses,latlong, on='address')

    test = pd.merge(test,adress_complete, on= 'ticket_id')
    train = pd.merge(train,adress_complete, on= 'ticket_id')

    # Eliminamos los nulos de 'hearing_date' 
    train.dropna(subset=['hearing_date'], inplace=True)
    test.dropna(subset=['hearing_date'], inplace=True)

    # calculamos la diferencia de fechas
    train['date_dif'] = (train.hearing_date - train.ticket_issued_date)
    train['date_dif'] = train['date_dif'].apply(lambda x : x.days)
    test['date_dif'] = (test.hearing_date - test.ticket_issued_date)
    test['date_dif'] = test['date_dif'].apply(lambda x : x.days)

    ### Que columnas seran significativas?

    # Eliminamos las columnas no significativas
    col_no_sig =["agency_name","inspector_name", "violator_name", "non_us_str_code",'violation_description','grafitti_status', 'violation_code', 'hearing_date','ticket_issued_date',
                # El total pagar es 'judgment_amount', asi que las siguientes pasan a ser no significativas
                'clean_up_cost', 'fine_amount','admin_fee','state_fee','late_fee','discount_amount',
                # Como normalizamos las direcciones y agregamos datos de latitud y longitud a cada ubicacion, estos pasan a ser no significativos
                'address','violation_street_number', 'violation_street_name', 'violation_zip_code','mailing_address_str_number','mailing_address_str_name', 
                'city', 'state', 'zip_code', 'country']

    train.drop(col_no_sig,axis = 1,inplace = True)
    test.drop(col_no_sig,axis = 1,inplace = True)

    ### Ahora con los datos mas limpios queda hacer un encoder de los datos

    # elimino las disposiciones diferentes de test

    disp_unique = train.disposition.unique()
    test_disp = test.disposition.unique()
    dif = [i for i in test_disp if i not in disp_unique]
    for i in dif:
        indexes = test[test.disposition == i].index.values
        test = test.drop(indexes, axis=0)

    # a disposition podemos hacerle un OneHotEncoder porque son 4 diferentes datos
    from sklearn.preprocessing import OneHotEncoder
    onehot_encoder = OneHotEncoder(sparse = False)

    disposition_train = train.disposition.values.reshape(-1,1)
    disposition_test = test.disposition.values.reshape(-1,1)
    onehot_encoder.fit(disposition_train)
    disposition_encoded_train = onehot_encoder.transform(disposition_train)
    disposition_encoded_test = onehot_encoder.transform(disposition_test)

    for i in range(0,len(onehot_encoder.categories_[0])):
        train[onehot_encoder.categories_[0][i]] = disposition_encoded_train[:,i]
        test[onehot_encoder.categories_[0][i]] = disposition_encoded_test[:,i]

    
    train.drop('disposition',axis = 1,inplace = True)
    test.drop('disposition',axis = 1,inplace = True)

    return train, test


Testeo del modelo

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

train, test = data_etl()
y = train['compliance']
X = train.drop('compliance',axis = 1)

# Para poder testear modelos mas rapido trabajare con la mitad de los datos del dataset train
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_test,y_test, random_state=0)

n = np.arange(1,5,1)
model = RandomForestClassifier()
param_grid = {'n_estimators': n, 'max_depth': n}
grid_search = GridSearchCV(model, param_grid, scoring="roc_auc",n_jobs=-1)
grid_result = grid_search.fit(X_train, y_train)
print('Grid best parameter: {}'.format(grid_result.best_params_))
print('Grid best score: {}'.format(grid_result.best_score_))

y_prob = grid_result.predict_proba(X_train)[:, 1]
print('Test set AUC: {}'.format(roc_auc_score(y_train, y_prob)))


Grid best parameter: {'max_depth': 4, 'n_estimators': 4}
Grid best score: 0.7886046151769832
Test set AUC: 0.7985134628399375


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


def blight_model():
    
    # Your code here

    train, test = data_etl()
    y = train['compliance']
    X = train.drop('compliance',axis = 1)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

    rfc = RandomForestClassifier(n_estimators=4, max_depth=4)
    rfc.fit(X_train, y_train)
    
    train_probs = rfc.predict_proba(X_train)[:, 1]
    test_probs = rfc.predict_proba(X_test)[:, 1]

    print('Train ROC AUC Score: {}'.format(roc_auc_score(y_train, train_probs)))
    print('Test ROC AUC  Score: {}'.format(roc_auc_score(y_test, test_probs)))

    y_prob = rfc.predict_proba(X_train)[:, 1]
    print('Test set AUC: {}'.format(roc_auc_score(y_train, y_prob)))

    test_pred = rfc.predict_proba(test)[:, 1]
    resultado = pd.Series(test_pred,index=test.ticket_id)
    
    return resultado

In [5]:
blight_model()

Train ROC AUC Score: 0.7921561435855546
Test ROC AUC  Score: 0.7866184957176152
Test set AUC: 0.7921561435855546


ticket_id
284932    0.097415
285362    0.025733
285361    0.097415
285338    0.097415
285346    0.097415
            ...   
376496    0.025733
376497    0.025733
376499    0.097415
376500    0.097415
369851    0.105943
Length: 58794, dtype: float64

El resultado deberia ser una series de length 61001. Como elimine algunos nulos quedo reducida. Probar sin eliminar esos valores.